In [1]:
import sys
from pathlib import Path

# Go two levels up to reach project root
project_root = Path.cwd().parents[1]  # from subfolder/ to notebook_folder/ to project_root/
scripts_path = project_root / "scripts"

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [2]:
import os
import contextlib

# Force matplotlib backend to avoid plotly attempts
os.environ["NP_PLOT_BACKEND"] = "matplotlib"

# Suppress stderr where the print happens
@contextlib.contextmanager
def suppress_stdout_stderr():
    with open(os.devnull, 'w') as fnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = fnull
        sys.stderr = fnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

# Wrap import
with suppress_stdout_stderr():
    from neuralprophet import NeuralProphet, set_random_seed, set_log_level

import pandas as pd
set_random_seed(0)
# Disable logging messages unless there is an error
set_log_level("ERROR")


In [ ]:
def modify(df, forecast=5):
    # figure out where 'yhat1' lives
    col_idx = df.columns.get_loc('yhat1')

    # loop 1…9
    for i in range(1, forecast):
        # compute which yhatN we need
        which = forecast - i + 1          # 10→1, 9→2, …, 2→9
        val   = df.iloc[-i][f'yhat{which}'] 
        # row number = -i (so i=1 → last row, i=9 → 9th-from-last)
        df.iat[-i, col_idx] = val

    # j = 0
    # for i in range(forecast+1, forecast+6):
    #     which = min(5, forecast - j +1)
    #     print(which)
    #     print(df.iloc[-i]['ds'])

    #     val = df.iloc[-i][f'yhat{which}']
    #     df.iat[-i, col_idx] = val
    #     j += 1

In [ ]:
# 1) Instantiate a NeuralProphet model that learns from the past 60 days (n_lags)
#    to forecast the next 10 days (n_forecasts), with trend + all seasonalities on.
m = NeuralProphet(
    n_lags=5,                 # look back this many days as inputs
    n_forecasts=2,            # predict this many days ahead
    yearly_seasonality=True,   # model an annual cycle
    weekly_seasonality=True,   # model day-of-week cycle
    daily_seasonality=False,    # if intra-day data; set False for daily only
    seasonality_mode='additive',
    n_changepoints=10,
    # you can also tweak trend flexibility:
    trend_reg=0.5,                  
    trend_reg_threshold=True,
    normalize='standardize',
    drop_missing=True,
    epochs=10
)
m.set_plotting_backend("plotly-static")
# m.add_country_holidays('VN')

In [ ]:
# 📂 Load dataset
file_path_1 = "../../ready_data/vn_index_data/cleaned_vn_index_data.csv"
df_1 = pd.read_csv(file_path_1)

# df_1.drop(columns=['Index'], inplace=True)

df = (
    df_1[['Date', 'VN_Index_Close']].rename(columns={
          'Date': 'ds',
          'VN_Index_Close': 'y'
      })
)

# convert the 'ds' column to datetime
df['ds'] = pd.to_datetime(df['ds'])

In [ ]:
df

In [ ]:
# 2) compute how many rows you need for validation
n_val = m.n_lags + m.n_forecasts  # must equal 60 + 10 = 70

# 3) split your DataFrame by row‐count
train_df = df.iloc[:-n_val]
val_df   = df.iloc[-n_val:]

# 4) fit using the fixed‐size validation set
metrics = m.fit(
    train_df,
    freq="B",
    validation_df=val_df,
    progress="plot",
)


In [ ]:
future = m.make_future_dataframe(df, periods=m.n_forecasts, n_historic_predictions=False)
forecast = m.predict(future, auto_extend=False, raw=False)

print(forecast)
modify(forecast, m.n_forecasts)
# Visualize the forecast
m.highlight_nth_step_ahead_of_each_forecast(step_number=1)
m.plot(forecast)

In [ ]:
forecast.tail(11)

In [ ]:
m.plot_components(forecast)

In [ ]:
forecast.tail(12)

In [ ]:
result = forecast[['ds', 'yhat1']].tail(5).rename(columns={
          'ds': 'Date',
          'yhat1': 'VN_Index_Close'
      })
result